In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix

In [2]:
adata = sc.read('/Users/dominik.klein/data/E14-E15_adata_filtered_rmDoublets_sctNormalized_WNN_embedded_initialAnno_new_annotation.h5ad')

In [3]:
adata.obsm["X_umap_wnn"] = adata.obsm["umap"]
adata.obsm["X_umap"] = adata.obsm["umap"]

In [4]:
adata.obsm["X_umap_GEX"][:,0] = -adata.obsm["X_umap_GEX"][:,0]  # make `direction` consistent

In [6]:
adata

AnnData object with n_obs × n_vars = 16953 × 14663
    obs: 'n_counts', 'sample', 'int_id', 'reporter', 'log_cell_probs', 'cell_barcodes_du', 'cell_barcodes_cr', 'cell_barcodes', 'log_counts', 'n_genes', 'log_genes', 'mt_frac', 'rp_frac', 'ambi_frac', 'total_counts_rank', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'leiden', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'log_nCount_ATAC', 'log_nFeature_ATAC', 'scrublet_scores', 'scrublet_doublets', 'scrublet_doublets_cat', 'dd_doublets', 'dd_scores', 'dd_log_p_values', 'dd_voting_average', 'dd_-log_p_values', 'dd_doublets_cat', 'pANN', 'DF_classifications_1', 'DF_classifications_2', 'pANN_SCT', 'DF_SCT_classifications_1', 'DF_SCT_classifications_2', 'df_doublets',

In [7]:
adata.obsp["connectivities_wnn"] = adata.obsp["connectivities"].copy()
adata.obsp["distances_wnn"] = adata.obsp["distances"].copy()

In [8]:
for layer in adata.layers:
    if not isinstance(adata.layers[layer], csc_matrix):
        adata.layers[layer] = csc_matrix(adata.layers[layer])

In [9]:
var_to_del = ['feature_types', 'genome', 'gene_ids-0', 'n_cells-0', 'ambient_genes-0', 'is_ambient-0', 'n_cells_by_counts-0', 
              'mean_counts-0', 'log1p_mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'log1p_total_counts-0', 'gene_ids-1', 
              'n_cells-1', 'ambient_genes-1', 'is_ambient-1', 'n_cells_by_counts-1', 'mean_counts-1', 'log1p_mean_counts-1', 'pct_dropout_by_counts-1', 
              'total_counts-1', 'log1p_total_counts-1', 'n_cells', 'sct.detection_rate', 'sct.gmean', 'sct.variance', 'sct.residual_variance', 'sct.variable', 
              'highly_variable']


In [10]:
obsm_to_del = ['X_pca', 'X_pca_wsnn', 'X_spca_wsnn', 'X_umap_wsnn', 'umap', 'umap_ATAC', 'umap_GEX', 'X_umap_wnn']

In [11]:
obs_cols_to_del = ['leiden_gex_graph', 'leiden_ATAC_graph', 'leiden_wnn_graph', 'umap_GEX_1', 'umap_GEX_2', 
                   'umap_ATAC_1', 'umap_ATAC_2', 'leiden_sub', 'leiden_sub1', 'leiden_sub2', 'leiden_sub3', 
                   'leiden_sub4', 'leiden_sub5', 'leiden_sub6', 'leiden_sub7', 'leiden_sub8', 'leiden_sub9', 
                   'leiden_sub10', 'leiden_sub11', 'leiden_sub12', 'leiden_sub13', 'leiden_sub14', 'leiden_sub15', 
                   'leiden_sub16', 'leiden_sub17', 'leiden_sub18', 'int_id', 'reporter', 'log_cell_probs', 
                   'cell_barcodes_du', 'cell_barcodes_cr', 'cell_barcodes', 'log_counts',
                  'scDblFinder.score', 'hybrid_class', 'hybrid_score', 'scDblFinder.class.sct', 'scDblFinder.score.sct', 'hybrid_class_sct', 'hybrid_score_sct', 'sdf_doublets', 'scds_doublets']


In [12]:
uns_to_del = ['dendrogram_leiden_sub1', 'dendrogram_leiden_sub12', 'dendrogram_leiden_sub14', 'dendrogram_leiden_sub15', 
              'dendrogram_leiden_sub16', 'dendrogram_leiden_sub17', 'dendrogram_leiden_sub18', 'dendrogram_leiden_sub2', 
              'dendrogram_leiden_sub3', 'dendrogram_leiden_sub4', 'dendrogram_leiden_sub5', 'dendrogram_leiden_sub6', 
              'dendrogram_leiden_sub7', 'dendrogram_leiden_sub8', 'dendrogram_leiden_sub9', 'final_doublets_cat_colors', 
              'leiden_ATAC_graph_colors', 'leiden_colors', 'leiden_gex_graph_colors', 'leiden_sub10_colors', 
              'leiden_sub11_colors', 'leiden_sub12_colors', 'leiden_sub13_colors', 'leiden_sub14_colors', 
              'leiden_sub15_colors', 'leiden_sub16_colors', 'leiden_sub17_colors', 'leiden_sub18_colors', 
              'leiden_sub1_colors', 'leiden_sub2_colors', 'leiden_sub3_colors', 'leiden_sub4_colors', 'leiden_sub5_colors', 
              'leiden_sub6_colors', 'leiden_sub7_colors', 'leiden_sub8_colors', 'leiden_sub9_colors', 'leiden_sub_colors', 
              'leiden_wnn_graph_colors']

In [13]:
varm_to_del = ['PCs', 'PCs_wsnn', 'SPCA_wsnn']

In [14]:
for el in var_to_del:
    del adata.var[el]

In [15]:
for el in obsm_to_del:
    del adata.obsm[el]

In [16]:
for col in obs_cols_to_del:
    del adata.obs[col]

In [17]:
for el in uns_to_del:
    del adata.uns[el]

In [18]:
for el in varm_to_del:
    del adata.varm[el]

In [19]:
adata.X = csc_matrix(np.round(np.exp(adata.layers["raw_counts"].A)-1))

In [20]:
del adata.layers["raw_counts"]

In [21]:
adata_lsi = sc.read("/Users/dominik.klein/data/E14-E15_adata_filtered_rmDoublets_lsi_peaks_to_adata.h5ad")

In [22]:
adata_lsi

AnnData object with n_obs × n_vars = 16978 × 1
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'doublet_calls', 'final_doublets', 'final_doublets_cat', 'nCount_peaks', 'nFeature_peaks', 'sample', 'peaks_snn_res.0.8', 'seurat_clusters'
    obsm: 'lsi_full', 'lsi_red'

In [23]:
def rename_index(x):
    prefix, barcode = x.split("_")
    if prefix == "E14-5":
        new_suffix="0"
    elif prefix == "E15-5":
        new_suffix="1"
    else:
        raise ValueError("no match")
    return barcode+"-"+new_suffix

In [24]:
adata_lsi.obs["old_index"] = adata_lsi.obs.index
adata_lsi.obs["index_adapted"] = adata_lsi.obs["old_index"].apply(rename_index)

In [25]:
adata_lsi_adapted = adata_lsi[adata_lsi.obs["index_adapted"].isin(adata.obs.index)]

In [26]:
adata_lsi_adapted.obs.set_index("index_adapted", inplace=True)

In [27]:
np.all(adata_lsi_adapted.obs.index == adata.obs.index)

True

In [28]:
adata.obsm["lsi_full"] = adata_lsi_adapted.obsm["lsi_full"]
adata.obsm["lsi_red"] = adata_lsi_adapted.obsm["lsi_red"]

In [29]:
adata

AnnData object with n_obs × n_vars = 16953 × 14663
    obs: 'n_counts', 'sample', 'n_genes', 'log_genes', 'mt_frac', 'rp_frac', 'ambi_frac', 'total_counts_rank', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'leiden', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'log_nCount_ATAC', 'log_nFeature_ATAC', 'scrublet_scores', 'scrublet_doublets', 'scrublet_doublets_cat', 'dd_doublets', 'dd_scores', 'dd_log_p_values', 'dd_voting_average', 'dd_-log_p_values', 'dd_doublets_cat', 'pANN', 'DF_classifications_1', 'DF_classifications_2', 'pANN_SCT', 'DF_SCT_classifications_1', 'DF_SCT_classifications_2', 'df_doublets', 'scDblFinder.class', 'doublet_calls', 'final_doublets', 'final_doublets_cat', 'batch', 'size_factors', 'mt_fra

In [30]:
adata_cr_peaks = sc.read("/Users/dominik.klein/data/E14-E15_adata_filtered_rmDoublets_cellranger_peaks.h5ad")


/Users/dominik.klein/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [31]:
adata_cr_peaks.obs["index_adapted"] = adata_cr_peaks.obs["Row.names"].apply(rename_index)

In [32]:
adata_cr_peaks_adapted = adata_cr_peaks[adata_cr_peaks.obs["index_adapted"].isin(adata.obs.index)]

In [33]:
adata_cr_peaks_adapted


View of AnnData object with n_obs × n_vars = 16953 × 228321
    obs: 'Row.names', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'doublet_calls', 'final_doublets', 'final_doublets_cat', 'nCount_peaks', 'nFeature_peaks', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'peaks.weight', 'sample', 'celltype_new', 'index_adapted'

In [34]:
adata_cr_peaks_adapted.obs.set_index("index_adapted", inplace=True)

In [35]:
np.all(adata_lsi_adapted.obs.index == adata.obs.index)

True

In [36]:
adata.obsm["cellranger_peaks"] = adata_cr_peaks_adapted.X

In [37]:
adata.write("/Users/dominik.klein/data/pancreas_multiome_2022.h5ad")